# Dynamische Arrays

Wir haben gesehen, wie wir die Methoden ```append``` und ```pop```, die ein Element am Ende eines Arrays anfügen, respektive entfernen, effizient implementieren können. Hier nochmals die Implementation:

In [2]:
class  Array:
    
    def __init__(self):
        self.data = [None] # list  simulates  block  of  memory
        self.size = 0  # index on next free element
        
    def append(self, elem):
        if self.size == len(self.data):    
            self.resize(len(self.data) * 2)
        self.data[self.size] = elem
        self.size  += 1        
    
    def pop(self):
        self.size -= 1
        item = self.data[self.size]; 
        if self.size > 0  and self.size == len(self.data) / 4:
            self.resize(int(len(self.data) / 2)) 

        return item; 

    def resize(self, numElements):
        newArray = [None] * numElements

        for i in  range(0, self.size):            
            newArray[i] = self.data[i]
        self.data = newArray
    
    def __str__(self):
        return str(self.data)
    
    def length(self):
        return self.size


Die folgenden Tests zeigen, dass die Implementation wie erwartet funktioniert:

In [6]:
a = Array()
for i in range(0, 10):
    a.append(i)
print("Array after append: " + str(a))
print("length: " +str(a.length()))

lastElement = a.pop()

print("lastElement " + str(lastElement))
print("Array after pop: " +str(a))
print("size after pop " +str(a.size))


Array after append: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None, None, None, None, None, None]
length: 10
lastElement 9
Array after pop: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, None, None, None, None, None, None]
size after pop 9


#### Experimente

Wir wollen nun testen, wie sich die Laufzeit der append Methode verhält. 

In [7]:
import timeit
import random

In [8]:
def createByAppend(n):
    a = Array()
    for i in range(0, n):
        a.append(i)
    return a;

In [9]:
for n in [10, 100, 10000, 100000]:
    t = timeit.timeit(lambda: createByAppend(n), number=1)
    print("Durchschnittliche Zeit für eine append Operation bei " + str(n) + " Elementen = " + str(t / n))

Durchschnittliche Zeit für eine append Operation bei 10 Elementen = 1.3499999994337485e-06
Durchschnittliche Zeit für eine append Operation bei 100 Elementen = 4.7100000017508136e-07
Durchschnittliche Zeit für eine append Operation bei 10000 Elementen = 3.9443000000005666e-07
Durchschnittliche Zeit für eine append Operation bei 100000 Elementen = 4.164560000000961e-07


Wie wir sehen, ist die Durchschnittliche Zeit pro ```append```-Operation konstant, unabhängig davon, wieviele Elemente wir einfügen. 

#### Miniübung 

* Vergleichen Sie die Laufzeit mit einer naiven Implementation, bei der das Array bei jedem Aufruf vergrössert wird.

### Ammortisierte Analyse: Array resizing

Im Folgenden schauen wir uns die ```append``` Operation noch auf theoretischer Ebene an. Der folgende Code dient zum illustrieren der Idee der amortisierten Analyse. Die Idee ist einfach: Wir schauen uns an, wieviele Arrayoperationen wir im Schnitt für eine Append Operation brauchen. Wenn diese Zahl mit einer Konstante nach oben abgeschätzt werden kann, ist die Laufzeit ammortisiert konstant:

Im folgenden Code zählen wir einfach die Arrayzugriffe und die Anzahl Append Operationen und geben diese dann aus.

In [157]:
class  ArrayWithAccounting:
   
    def __init__(self):
        self.data = [None] 
        self.size = 0
                
        self.numberOfArrayAccesses = 0
        self.numberOfAppendOperations = 0
        
    def append(self, elem):
        
        self.numberOfAppendOperations += 1
        
        if self.size == len(self.data):            
            newCapacity = len(self.data) * 2
            self.resize(newCapacity)                        
                            
        self.data[self.size] = elem
        self.numberOfArrayAccesses += 1
        
        self.size  += 1        

  
    def resize(self, numElements):
        # Ein neues Array anzulegen braucht 1 Array Zugriffe pro Element
        newArray = [None] * numElements
        self.numberOfArrayAccesses += numElements
        
        for i in  range(0, self.size):
            # das kopieren brauch 2 Array Zugriffe
            newArray[i] = self.data[i]            
            self.numberOfArrayAccesses += 2

        self.data = newArray
                
    def ratioAppendArrayAccess(self):
        return self.numberOfArrayAccesses / self.numberOfAppendOperations
        
    
    def __str__(self):
        return str(self.data)


Wenn wir diesen Code Ausführen, dann sehen wir, dass wir nach jeder Append Operation immer eine positive Anzahl Tokens haben. 

In [160]:
a = ArrayWithAccounting()
for i in range(1, 100):
    a.append(i)
    print(a.ratioAppendArrayAccess())
    

1.0
3.0
5.0
4.0
6.6
5.666666666666667
5.0
4.5
7.666666666666667
7.0
6.454545454545454
6.0
5.615384615384615
5.285714285714286
5.0
4.75
8.294117647058824
7.888888888888889
7.526315789473684
7.2
6.904761904761905
6.636363636363637
6.391304347826087
6.166666666666667
5.96
5.769230769230769
5.592592592592593
5.428571428571429
5.275862068965517
5.133333333333334
5.0
4.875
8.636363636363637
8.411764705882353
8.2
8.0
7.8108108108108105
7.631578947368421
7.461538461538462
7.3
7.146341463414634
7.0
6.8604651162790695
6.7272727272727275
6.6
6.478260869565218
6.361702127659575
6.25
6.142857142857143
6.04
5.9411764705882355
5.846153846153846
5.754716981132075
5.666666666666667
5.581818181818182
5.5
5.421052631578948
5.344827586206897
5.271186440677966
5.2
5.131147540983607
5.064516129032258
5.0
4.9375
8.815384615384616
8.696969696969697
8.582089552238806
8.470588235294118
8.36231884057971
8.257142857142858
8.154929577464788
8.055555555555555
7.958904109589041
7.864864864864865
7.773333333333333
7.

#### Übung: 

* Was ist das grösste Verhältnis, welches Sie in einer langen Sequenz von Append Operationen sehen?
* Wie ist das Verhältnis, wenn die Zahl eine Zweierpotenz ist? Warum?